# Simple RAG 소개

Retrieval-Augmented Generation(RAG)은 정보 검색과 생성형 언어 모델을 결합한 접근 방식이에요. 이 방식은 외부 지식을 모델에 활용할 수 있게 해줘서, 더 정확하고 사실 기반의 답변을 생성하는 데 도움이 됩니다.

Simple RAG의 기본 흐름은 다음과 같습니다.

1. **데이터 수집 및 전처리**: 텍스트 데이터를 불러오고, 사용할 수 있는 형태로 정리합니다.
2. **청킹(Chunking)**: 검색 효율을 높이기 위해 데이터를 적당한 크기의 조각으로 나눕니다.
3. **임베딩 생성**: 조각낸 텍스트를 임베딩 모델을 통해 숫자 벡터로 바꿔줍니다.
4. **Semantic Search**: 사용자의 질문과 관련 있는 텍스트 조각을 의미 기반으로 찾아냅니다.
5. **응답 생성**: 검색된 내용을 바탕으로 언어 모델이 최종 답변을 만들어냅니다.

해당 노트북에서는 Simple RAG 방식의 기본 구현을 다루고, 응답 품질을 평가한 뒤, 더 나은 성능을 위한 개선 방향도 함께 살펴봅니다.


## 환경 설정하기
필요한 라이브러리를 가져오는 것부터 시작합니다.

In [ ]:
import os
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv
import json

# .env 파일 로드
load_dotenv()

API_KEY = os.environ.get('api_key')
gemini_API_KEY = os.environ.get('gemini_api_key')

## PDF 파일에서 텍스트 추출하기
RAG를 구현하려면 먼저 텍스트 데이터 소스가 필요합니다. 저는 gemini를 이용해 pdf에서 텍스트를 추출하는 방식을 사용합니다.

In [2]:
import google.generativeai as genai

def extract_text_from_pdf(pdf_path):
    # API 키 설정
    genai.configure(api_key=gemini_API_KEY)
    client = genai.GenerativeModel('gemini-2.0-flash-lite')

    # PDF 파일 업로드
    with open(pdf_path, "rb") as file:
        file_data = file.read()


    prompt = "Extract all text from the provided PDF file."
    response = client.generate_content([
        {"mime_type": "application/pdf", "data": file_data},
        prompt
    ],generation_config={
            "max_output_tokens": 8192  # 최대 출력 토큰 수 설정 (예: 8192 토큰, 약 24,000~32,000자)
    })
    return response.text

In [3]:
# 이미 text 파일로 저장되어 있다면 load_text_file 함수를 사용하면 됩니다.
def load_text_file(pdf_path):

    # text 파일 로드
    with open(pdf_path, "r", encoding="utf-8") as txt_file:
        text = txt_file.read()

    return text

## 추출된 텍스트 청크 분할하기
추출된 텍스트가 있으면 검색 정확도를 높이기 위해 더 작고 겹치는 청크로 분할합니다.

In [4]:
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 겹치는 n개의 문자 세그먼트로 청크합니다.

    Args:
    text (str): 청크할 텍스트입니다.
    n (int): 각 청크의 문자 수입니다.
    overlap (int): 청크 간 겹치는 문자 수입니다.

    Returns:
    List[str]: 청크된 텍스트 리스트입니다.
    """
    chunks = []  # 청크된 텍스트를 저장할 빈 리스트를 초기화합니다.
    
    # (n - overlap) 단계로 텍스트를 반복합니다.
    for i in range(0, len(text), n - overlap):
        # 인덱스 i부터 i + n까지의 텍스트를 청크 리스트에 추가합니다.
        chunks.append(text[i:i + n])

    return chunks  

## OpenAI API 클라이언트 설정하기

In [5]:
from openai import OpenAI

client_openai = OpenAI(api_key = API_KEY)

## PDF 파일에서 텍스트 추출 및 청크 분할하기
이제 PDF를 로드하고 텍스트를 추출한 후 청크로 분할합니다.

In [ ]:
# PDF 파일의 경로를 정의합니다.
# pdf_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.pdf"

# PDF 파일에서 텍스트 추출
# extracted_text = extract_text_from_pdf(pdf_path)


# text 파일의 경로를 정의합니다.
text_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"

# text 파일로드
extracted_text = load_text_file(text_path)

# 추출된 텍스트를 512개의 문자 세그먼트로 청크하고, 200개의 문자를 겹치게 합니다.
text_chunks = chunk_text(extracted_text, 512, 50)

# 청크된 텍스트의 개수를 출력합니다.
print("Number of text chunks:", len(text_chunks))

# 청크된 텍스트의 첫 번째 조각을 출력합니다.
print("\nFirst text chunk:")
print(text_chunks[0])

## 분할된 청크의 임베딩 생성
임베딩은 텍스트를 숫자 벡터로 변환하여 효율적인 유사도 검색을 가능하게 합니다.

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
def create_embeddings(embedding_model, texts, device='cuda', batch_size=16):
    """
    SentenceTransformer 모델을 사용하여 지정된 텍스트에 대한 임베딩을 생성합니다.

    Args:
        embedding_model: 임베딩을 생성할 SentenceTransformer 모델입니다.
        texts (list): 임베딩을 생성할 입력 텍스트 리스트입니다.
        device (str): 모델을 실행할 장치 ('cuda' for GPU, 'cpu' for CPU).
        batch_size (int): 인코딩을 위한 배치 크기입니다.

    Returns:
        np.ndarray: 모델에 의해 생성된 임베딩입니다.
    """
    # 모델이 지정된 장치에 있는지 확인합니다.
    embedding_model = embedding_model.to(device)
    
    # 지정된 배치 크기로 임베딩을 생성합니다.
    embeddings = embedding_model.encode(
        texts,
        device=device,
        batch_size=batch_size,  # 메모리 사용량을 줄이기 위해 더 작은 배치 크기를 사용합니다.
        show_progress_bar=True  # 인코딩 진행 상태를 모니터링하기 위한 진행 표시줄을 표시합니다.
    )
    
    return embeddings

# GPU 사용 가능 여부를 확인합니다.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# 모델을 로드합니다.
model = "BAAI/bge-m3"
embedding_model = SentenceTransformer(model)


# 배치 크기를 줄여 임베딩을 생성합니다.
embeddings = create_embeddings(embedding_model, text_chunks, device=device, batch_size=4)

print(f"Generated {len(embeddings)} sentence embeddings.")

## 시맨틱 검색 수행하기
코사인 유사도를 구현하여 사용자 쿼리에 가장 관련성이 높은 텍스트 청크를 찾습니다.

In [10]:
def cosine_similarity(vec1, vec2):
    """
    두 벡터 간의 코사인 유사도를 계산합니다.

    Args:
    vec1 (np.ndarray): 첫 번째 벡터입니다.
    vec2 (np.ndarray): 두 번째 벡터입니다.

    Returns:
    float: 두 벡터 간의 코사인 유사도입니다.
    """
    # 두 벡터의 내적을 계산하고 두 벡터의 크기의 곱으로 나눕니다.
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [11]:
def semantic_search(query, text_chunks, embeddings, k=5):
    """
    주어진 쿼리와 임베딩을 사용하여 텍스트 청크에서 시맨틱 검색을 수행합니다.

    Args:
    query (str): 시맨틱 검색을 위한 쿼리입니다.
    text_chunks (List[str]): 검색할 텍스트 청크 리스트입니다.
    embeddings (List[dict]): 텍스트 청크에 대한 임베딩 리스트입니다.
    k (int): 상위 k개의 관련 텍스트 청크를 반환합니다. 기본값은 5입니다.

    Returns:
    List[str]: 쿼리와 가장 유사한 상위 k개의 텍스트 청크 리스트입니다.
    """
    # 쿼리에 대한 임베딩을 생성합니다.
    query_embedding = create_embeddings(embedding_model, [query])[0]
    similarity_scores = []  # 유사도 점수를 저장할 빈 리스트를 초기화합니다.

    # 쿼리 임베딩과 각 텍스트 청크 임베딩 간의 유사도 점수를 계산합니다.
    for i, chunk_embedding in enumerate(embeddings):
        similarity_score = cosine_similarity(np.array(query_embedding), np.array(chunk_embedding))
        similarity_scores.append((i, similarity_score))  # 인덱스와 유사도 점수를 리스트에 추가합니다.

    # 유사도 점수를 내림차순으로 정렬합니다.
    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    # 상위 k개의 유사한 텍스트 청크의 인덱스를 추출합니다.
    top_indices = [index for index, _ in similarity_scores[:k]]
    # 상위 k개의 관련 텍스트 청크를 반환합니다.
    return [text_chunks[index] for index in top_indices]


## 추출된 청크에서 쿼리 실행하기

In [ ]:
import pandas as pd
# 평가 데이터 로드하기
df = pd.read_csv('./data_creation/rag_val_new_post.csv')
df.head()

In [ ]:
# 평가 데이터에서 첫 번째 쿼리를 추출합니다.
query = df['query'][0]

# 시맨틱 검색을 수행하여 쿼리와 가장 연관성이 높은 상위 2개의 텍스트 청크를 찾습니다.
top_chunks = semantic_search(query, text_chunks, embeddings, k=3)

# 쿼리 출력
print("Query:", query)

# 쿼리와 연관있는 2개의 chunk 출력
for i, chunk in enumerate(top_chunks):
    print(f"Context {i + 1}:\n{chunk}\n=====================================")

## 검색된 청크를 기반으로 response 생성하기

In [15]:
def generate_response(system_prompt, user_message ,model_name='gpt-4.1-nano'):
    
    response = client_openai.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},# Define the system prompt for the AI assistant
            {"role": "user", "content": user_message}
        ],
        temperature=0.1,
        top_p=0.9,
        max_tokens=1024,
    )
    # print(response.choices[0].message.content)

    return response

# 검색된 청크를 기반으로 사용자 프롬프트를 생성합니다.
user_prompt = "\n".join([f"Context {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
user_prompt = f"{user_prompt}\n질문: {query}"


# 시스템 프롬프트를 정의합니다.
system_prompt = "당신은 제공된 Context에 기반하여 답변하는 AI 어시스턴트입니다. 답변이 컨텍스트에서 직접 도출될 수 없는 경우, 다음 문장을 사용하세요: '해당 질문에 답변할 충분한 정보가 없습니다.'"
# 응답 생성
ai_response = generate_response(system_prompt, user_prompt)

In [ ]:
print(ai_response.choices[0].message.content)

## 생성 응답 평가하기
생서 응답을 예상 답변과 비교하여 점수를 부여합니다.

In [ ]:
# 평가 시스템의 시스템 프롬프트를 정의합니다.
evaluate_system_prompt = "You are an intelligent evaluation system tasked with assessing the AI assistant's responses. If the AI assistant's response is very close to the true response, assign a score of 1. If the response is incorrect or unsatisfactory in relation to the true response, assign a score of 0. If the response is partially aligned with the true response, assign a score of 0.5."

# 사용자 쿼리, AI 응답, 실제 응답, 평가 시스템 프롬프트를 결합하여 평가 프롬프트를 생성합니다.
evaluation_prompt = f"User Query: {query}\nAI Response:\n{ai_response.choices[0].message.content}\nTrue Response: {df['generation_gt'][0]}\n{evaluate_system_prompt}"

# 평가 시스템 프롬프트와 평가 프롬프트를 사용하여 평가 응답을 생성합니다.
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt,'gpt-4.1-mini')

# 평가 응답을 출력합니다.
print(evaluation_response.choices[0].message.content)

## 전체 추론 및 평가

In [ ]:
# 생성된 답변과 실제 답변을 비교하여 점수를 부여합니다.
result = []

for i in range(len(df)):
    query = df['query'][i]
    top_chunks = semantic_search(query, text_chunks, embeddings, k=3)

    # AI 어시스턴트의 시스템 프롬프트를 정의합니다.
    system_prompt = "당신은 제공된 Context에 기반하여 답변하는 AI 어시스턴트입니다. 답변이 컨텍스트에서 직접 도출될 수 없는 경우, 다음 문장을 사용하세요: '해당 질문에 답변할 충분한 정보가 없습니다.'"
        
    # 검색된 청크를 기반으로 사용자 프롬프트를 생성합니다.
    user_prompt = "\n".join([f"Context {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
    user_prompt = f"{user_prompt}\n질문: {query}"

    # response 생성
    ai_response = generate_response(system_prompt, user_prompt, 'gpt-4.1-mini')

    print(ai_response.choices[0].message.content)
    
    # 평가 프롬프트를 정의합니다.
    evaluate_system_prompt = "You are an intelligent evaluation system tasked with assessing the AI assistant's responses. If the AI assistant's response is very close to the true response, assign a score of 1. If the response is incorrect or unsatisfactory in relation to the true response, assign a score of 0. If the response is partially aligned with the true response, assign a score of 0.5."

    # 사용자 쿼리, AI response, 실제 응답, 평가 시스템 프롬프트를 결합하여 평가 프롬프트를 생성합니다.
    evaluation_prompt = f"User Query: {query}\nAI Response:\n{ai_response.choices[0].message.content}\nTrue Response: {df['generation_gt'][0]}\n{evaluate_system_prompt}"

    # 평가 응답 생성
    evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt,'gpt-4.1-mini')

    # 평가 응답을 출력합니다.
    print(evaluation_response.choices[0].message.content)
    result.append(evaluation_response.choices[0].message.content)


## 점수 계산

In [ ]:
import numpy as np
result = [float(x.replace('\n',''))for x in result]
print(np.mean(result))